In [2]:
import keras_nlp

Using TensorFlow backend


In [3]:
# Unbatched input.
tokenizer = keras_nlp.models.AlbertTokenizer.from_preset("albert_base_en_uncased")

In [4]:
tokenizer("The quick brown fox jumped.")

<tf.Tensor: shape=(8,), dtype=int32, numpy=array([  13,    1,  438, 2231,  886, 2385, 4298,    9])>

In [5]:
# Batched input.
tokenizer(["The quick brown fox jumped.", "The fox slept."])

<tf.RaggedTensor [[13, 1, 438, 2231, 886, 2385, 4298, 9], [13, 1, 438, 2385, 7132, 9]]>

In [6]:
# Detokenization.
tokenizer.detokenize(tokenizer("The quick brown fox jumped."))

<tf.Tensor: shape=(), dtype=string, numpy=b' \xe2\x81\x87 he quick brown fox jumped.'>

In [7]:
# Custom vocabulary.
import io
import tensorflow as tf
import sentencepiece
#from keras_nlp import sentencepiece
bytes_io = io.BytesIO()
ds = tf.data.Dataset.from_tensor_slices(["The quick brown fox jumped."])
sentencepiece.SentencePieceTrainer.train(
    sentence_iterator=ds.as_numpy_iterator(),
    model_writer=bytes_io,
    vocab_size=10,
    model_type="WORD",
    pad_id=0,
    unk_id=1,
    bos_id=2,
    eos_id=3,
    pad_piece="<pad>",
    unk_piece="<unk>",
    bos_piece="[CLS]",
    eos_piece="[SEP]",
    user_defined_symbols="[MASK]",
)

In [8]:
tokenizer = keras_nlp.models.AlbertTokenizer(
    proto=bytes_io.getvalue(),
)
tokenizer("The quick brown fox jumped.")

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([5, 9, 6, 7, 8])>

In [9]:
preprocessor = keras_nlp.models.AlbertPreprocessor.from_preset(
    "albert_base_en_uncased"
)

In [10]:
# Tokenize and pack a single sentence.
preprocessor("The quick brown fox jumped.")

{'token_ids': <tf.Tensor: shape=(512,), dtype=int32, numpy=
 array([   2,   13,    1,  438, 2231,  886, 2385, 4298,    9,    3,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,   

In [11]:
# Tokenize a batch of single sentences.
preprocessor(["The quick brown fox jumped.", "Call me Ishmael."])

{'token_ids': <tf.Tensor: shape=(2, 512), dtype=int32, numpy=
 array([[ 2, 13,  1, ...,  0,  0,  0],
        [ 2, 13,  1, ...,  0,  0,  0]])>,
 'segment_ids': <tf.Tensor: shape=(2, 512), dtype=int32, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])>,
 'padding_mask': <tf.Tensor: shape=(2, 512), dtype=bool, numpy=
 array([[ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False]])>}

In [12]:
import numpy as np

In [13]:
input_data = {
    "token_ids": np.ones(shape=(1, 12), dtype="int32"),
    "segment_ids": np.array([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0]]),
    "padding_mask": np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]]),
}

# Randomly initialized ALBERT encoder
model = keras_nlp.models.AlbertBackbone(
    vocabulary_size=30000,
    num_layers=12,
    num_heads=12,
    num_groups=1,
    num_inner_repetitions=1,
    embedding_dim=128,
    hidden_dim=768,
    intermediate_dim=3072,
    max_sequence_length=12,
)
output = model(input_data)
output

{'sequence_output': <tf.Tensor: shape=(1, 12, 768), dtype=float32, numpy=
 array([[[-0.90249944, -1.7424964 ,  0.48982424, ...,  0.5020634 ,
           0.07291413,  2.2348678 ],
         [-0.56565154, -1.9457185 ,  0.84350604, ...,  0.18744577,
          -0.44964558,  2.3078506 ],
         [-0.82057583, -2.1958494 ,  0.47708926, ...,  0.87245214,
           0.08581122,  1.6888585 ],
         ...,
         [-1.142025  , -1.9482069 ,  0.31074753, ...,  0.6979596 ,
          -0.05349603,  1.3303281 ],
         [-0.55054593, -2.0325682 ,  1.0123701 , ...,  0.46946406,
          -0.6558784 ,  1.9517909 ],
         [-0.40875772, -1.8813509 ,  0.8480543 , ...,  0.36955604,
          -1.0256791 ,  2.107257  ]]], dtype=float32)>,
 'pooled_output': <tf.Tensor: shape=(1, 768), dtype=float32, numpy=
 array([[ 1.63835377e-01, -6.00778401e-01, -2.41029114e-01,
          4.42382246e-01,  5.23522198e-01, -6.44949257e-01,
          1.76148340e-01,  6.20967329e-01, -3.22868794e-01,
         -3.58818442e

In [15]:
import keras

In [16]:
features = ["The quick brown fox jumped.", "I forgot my homework."]
labels = [0, 3]

# Pretrained classifier.
classifier = keras_nlp.models.AlbertClassifier.from_preset(
    "albert_base_en_uncased",
    num_classes=4,
)
classifier.fit(x=features, y=labels, batch_size=2)
classifier.predict(x=features, batch_size=2)

# Re-compile (e.g., with a new learning rate).
classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(5e-5),
    jit_compile=True,
)
# Access backbone programmatically (e.g., to change `trainable`).
classifier.backbone.trainable = False
# Fit again.
classifier.fit(x=features, y=labels, batch_size=2)

1/1 [==============================] - 123s 123s/step - loss: 1.2114


In [17]:
features = {
    "token_ids": np.ones(shape=(2, 12), dtype="int32"),
    "segment_ids": np.array([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0]] * 2),
    "padding_mask": np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]] * 2),
}
labels = [0, 3]

# Pretrained classifier without preprocessing.
classifier = keras_nlp.models.AlbertClassifier.from_preset(
    "albert_base_en_uncased",
    num_classes=4,
    preprocessor=None,
)
classifier.fit(x=features, y=labels, batch_size=2)

1/1 [==============================] - 176s 176s/step - loss: 1.5074 - sparse_categorical_accuracy: 0.0000e+00


In [18]:
# Custom backbone and vocabulary.

In [19]:
features = ["The quick brown fox jumped.", "I forgot my homework."]
labels = [0, 3]

bytes_io = io.BytesIO()
ds = tf.data.Dataset.from_tensor_slices(features)
sentencepiece.SentencePieceTrainer.train(
    sentence_iterator=ds.as_numpy_iterator(),
    model_writer=bytes_io,
    vocab_size=10,
    model_type="WORD",
    pad_id=0,
    unk_id=1,
    bos_id=2,
    eos_id=3,
    pad_piece="<pad>",
    unk_piece="<unk>",
    bos_piece="[CLS]",
    eos_piece="[SEP]",
    user_defined_symbols="[MASK]",
)
tokenizer = keras_nlp.models.AlbertTokenizer(
    proto=bytes_io.getvalue(),
)
preprocessor = keras_nlp.models.AlbertPreprocessor(
    tokenizer=tokenizer,
    sequence_length=128,
)
backbone = keras_nlp.models.AlbertBackbone(
    vocabulary_size=tokenizer.vocabulary_size(),
    num_layers=4,
    num_heads=4,
    hidden_dim=256,
    embedding_dim=128,
    intermediate_dim=512,
    max_sequence_length=128,
)
classifier = keras_nlp.models.AlbertClassifier(
    backbone=backbone,
    preprocessor=preprocessor,
    num_classes=4,
)
classifier.fit(x=features, y=labels, batch_size=2)

1/1 [==============================] - 54s 54s/step - loss: 1.3739 - sparse_categorical_accuracy: 0.0000e+00


In [20]:
# Instantiate AlbertClassifier model from preset architecture and weights.

In [22]:
from keras_nlp.models import AlbertClassifier

In [28]:
# Load architecture and weights from preset
model = AlbertClassifier.from_preset("albert_base_en_uncased",num_classes=4)

# Load randomly initialized model from preset architecture
model = AlbertClassifier.from_preset("albert_base_en_uncased",num_classes= 4,load_weights=False)